In [1]:
import matplotlib
import pathlib
import mne
import os
import numpy as np

In [2]:
# LOAD RAW:
matplotlib.use('Qt5Agg')
data_dir = 'D:/raw_/original data/EXP/second time'
set_files = [f for f in os.listdir(data_dir) if f.endswith('.set')]

for set_file in set_files:
    set_path = os.path.join(data_dir,set_file)
    raw = mne.io.read_raw_eeglab(set_path,uint16_codec='latin1')
    # 上一行遇到问题了，记一下
    print(f"Loaded data from {set_path}")

Reading D:\raw_\original data\EXP\second time\resting-state-2001-2.edat3_204232_raw.fdt
Loaded data from D:/raw_/original data/EXP/second time\resting-state-2001-2.edat3_204232_raw.set
Reading D:\raw_\original data\EXP\second time\resting-state-2011-2.edat3_182210_raw.fdt
Loaded data from D:/raw_/original data/EXP/second time\resting-state-2011-2.edat3_182210_raw.set
Reading D:\raw_\original data\EXP\second time\resting-state-2012-2.edat3_193510_raw.fdt
Loaded data from D:/raw_/original data/EXP/second time\resting-state-2012-2.edat3_193510_raw.set
Reading D:\raw_\original data\EXP\second time\resting-state-2013-2.edat3_230531_raw.fdt
Loaded data from D:/raw_/original data/EXP/second time\resting-state-2013-2.edat3_230531_raw.set
Reading D:\raw_\original data\EXP\second time\resting-state-2014-2.edat3_309030_raw.fdt
Loaded data from D:/raw_/original data/EXP/second time\resting-state-2014-2.edat3_309030_raw.set
Reading D:\raw_\original data\EXP\second time\resting-state-2015-2.edat3_21

In [3]:
# 为了计算后续crop长度&resampling目标频率，读取
freq = raw.info['sfreq']
n_samples = raw.n_times/freq
n_samples # 采样总数

844.0

In [12]:
# Delete non-eeg channels

non_eeg_chan = [
'ECG', 'HEOR', 'HEOL', 'VEOU', 'VEOL', 'ECGM_48654652']

raw.drop_channels(non_eeg_chan)

raw.info['ch_names']
raw.plot()

ValueError: Channel(s) ECG, HEOR, HEOL, VEOU, VEOL, ECGM_48654652 not found, nothing dropped.

In [13]:
# NO CHANNEL LOCATIONS IN EEG FILE

montage = mne.channels.make_standard_montage("standard_1005")
print(montage)

raw.set_montage(montage)
fig = raw.plot_sensors(show_names=True)

#定位完成

<DigMontage | 0 extras (headshape), 0 HPIs, 3 fiducials, 343 channels>


In [7]:
# Read events & crop
events, event_id = mne.events_from_annotations(raw)
# 源数据里没有语义化events，自己用词典
# event_id = {'Openeyes':events[1]['12'],'Closeeyes':events[1]['14']}


# 直接把tuple的描述字典赋值给event_id
# 每隔一秒建立一个event
event_id = 1  # 事件ID
duration = 1  # 事件持续时间/秒
events = mne.make_fixed_length_events(raw, event_id, start=0, duration=duration, overlap=0)
events

Used Annotations descriptions: ['12', '13', '14', '15']


array([[     0,      0,      1],
       [  1000,      0,      1],
       [  2000,      0,      1],
       ...,
       [841000,      0,      1],
       [842000,      0,      1],
       [843000,      0,      1]])

In [14]:
# 创建Epochs对象
reject_criteria = dict(eeg = 100e-6)

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=-1,
                    tmax=1,
                    reject=reject_criteria,
                    preload=True)

# raw_cropped = raw.crop(tmin=t_start/1000,tmax=t_end/1000)
# raw_cropped.plot()

Not setting metadata
844 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 844 events and 2001 original time points ...
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F4', 'F6', 'F7', 'FC5', 'C3', 'CP1', 'TP7', 'PO5', 'PO7', 'PO8', 'O1', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F3', 'F4', 'F5', 'F6', 'F7', 'FC5', 'FT7', 'C3', 'T7', 'T8', 'CP1', 'CP5', 'TP7', 'TP8', 'P8', 'PO7', 'PO8', 'Oz', 'O1', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC5', 'FT7', 'FT8', 'T7', 'T8', 'CP1', 'TP7', 'TP8', 'P8', 'PO7', 'PO8', 'O2']
    Rejecting  epoch based on EEG : ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'FC5', 'FT7', 'FT8', 'T7', 'T8', 'TP8', 'PO7', 'PO8', 'O1', 'O2']
    Rejec

In [ ]:
# 上面drop的太多了

In [16]:
# resampling
target_sfreq = 500

raw_resampled = raw.copy().resample(sfreq=target_sfreq)
raw_resampled.plot_psd(fmax=250)

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


<MNELineFigure size 1000x350 with 2 Axes>

In [17]:
raw_resampled.plot()

Channels marked as bad:
none


In [18]:
# filtering
raw_filtered=raw_resampled.copy().filter(l_freq=1,h_freq=100)
raw_filtered
raw_filtered.plot()
raw_filtered.plot_psd(fmax=250)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


<MNELineFigure size 1000x350 with 2 Axes>

Channels marked as bad:
none


In [19]:
# ICA
raw_ica = raw_filtered.copy()
ica = mne.preprocessing.ICA(n_components= 20)
ica.fit(raw_ica)
raw_ica.load_data()
ica.plot_sources(raw_ica)
ica.plot_components()

Fitting ICA to data using 59 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 9.5s.
Creating RawArray with float64 data, n_channels=20, n_times=422000
    Range : 0 ... 421999 =      0.000 ...   843.998 secs
Ready.


<MNEFigure size 975x967 with 20 Axes>

In [20]:
# re-ref based on average channel(?
raw_ref = raw_ica.copy()
raw_ref.load_data()
raw_ref.set_eeg_reference(ref_channels="average", projection=False)

raw_ref.plot()

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


Channels marked as bad:
none


In [21]:
epochs.plot()

In [22]:
raw_ref.plot_psd()

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


<MNELineFigure size 1000x350 with 2 Axes>

Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
